### 사전 준비 사항 

#### (1) uv add (터미널)

```bash
uv add pdfplumber sentence-transformers faiss-cpu numpy torch python-dotenv openai
```

#### (2) .env 파일 세팅
```bash
OPENAI_API_KEY = ""
HF_TOKEN = ""
```

#### (3) pdf 파일 세팅
pdf 파일 100개를 `data/raw/files` 에 위치합니다.  
성능 개선을 위한 테스트에서는 전체를 대상으로 인덱싱하지 않고, 5개 인덱싱 코드 실행 권장(주석 해제)

In [25]:
from preprocess.pp_basic import docs, BASE_DIR, EVAL_DIR, GOLD_EVIDENCE_CSV, GOLD_FIELDS_JSONL, RAW_DIR
import preprocess.pp_v4 as pp

In [26]:
import pdfplumber
import os
from pathlib import Path
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import torch

In [27]:
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()
client = OpenAI()
GEN_MODELS = ["gpt-5-mini", "gpt-5-nano"]

In [28]:
# 임베딩 및 인덱스 만들기
# 한국어 임베딩 모델
embed_model_name = "nlpai-lab/KoE5"
embed_model = SentenceTransformer(embed_model_name)

Loading weights: 100%|██████████| 391/391 [00:00<00:00, 530.74it/s, Materializing param=pooler.dense.weight]                               


### 파서/cleanup/SYSTEM 추가

In [29]:
import re

SYSTEM = (
  "너는 RFP 문서에서 질문의 답을 '추출'한다.\n"
  "절대 문서 원문을 길게 복사하지 마라.\n"
  "출력은 오직 'field<TAB>answer' 형식의 줄들만 허용한다.\n"
  "field는 제공된 목록만 사용한다.\n"
  "각 field는 반드시 정확히 한 줄씩 출력한다.\n"
)

def parse_field_answers_relaxed(text: str, fields: list[str]) -> dict[str, str]:
    """
    TSV(탭) 우선, 실패 시 콜론/대시/등호도 허용.
    """
    m = {f: "" for f in fields}
    if not text:
        return m

    field_set = set(fields)

    for line in text.splitlines():
        line = line.strip()
        if not line:
            continue

        # 1) field<TAB>answer
        if "\t" in line:
            k, v = line.split("\t", 1)
        # 2) field: answer
        elif ":" in line:
            k, v = line.split(":", 1)
        # 3) field - answer / field = answer
        elif " - " in line:
            k, v = line.split(" - ", 1)
        elif " = " in line:
            k, v = line.split(" = ", 1)
        else:
            continue

        k = k.strip().lstrip("-•").strip()
        v = v.strip()
        if k in field_set:
            m[k] = v

    return m

def cleanup_chunk(t: str) -> str:
    """
    PDF 텍스트 깨짐(문자 반복) 완화용. 너무 공격적이면 끄면 됨.
    """
    t = re.sub(r"(.)\1{3,}", r"\1", t)  # 같은 문자 4번 이상 반복 -> 1번
    return t

In [30]:
# RFP 분석용 프롬프트
RFP_PROMPT = """
너는 정부·공공기관 제안요청서(RFP)를 분석하는 전문가다.
아래 컨텍스트는 하나의 정부 RFP 문서에서 추출된 내용이다.

[분석 규칙]
- 추측 금지, 문서에 명시된 내용만 사용
- 문서에 없으면 반드시 "NOT_FOUND"
- **출력은 질문 개수와 동일한 줄 수**
- **각 줄에는 답변 텍스트만 작성**
- 번호, 질문 문장, '답:', 기호, 설명을 절대 포함하지 말 것

[질문 목록]
{questions}

[컨텍스트]
{context}

질문 순서대로 답변만 한 줄씩 출력하라.
"""

### 배치 질문 응답

질문 리스트를 한 번에 임베딩/검색하고, top-k 청크를 합쳐 LLM에 넣어 답변을 생성합니다.
`return_indices=True`일 때는 검색된 청크 인덱스도 반환합니다.

In [31]:
def build_query_prompt(queries):
    return "\n".join(
        f"{i}. {q}"
        for i, (_, q) in enumerate(queries, start=1)
    )

In [32]:
def parse_field_answers_tsv(text: str, fields: list[str]) -> dict[str, str]:
    """
    모델 출력이 'field<TAB>answer' 형식이라고 가정.
    누락된 field는 "" 처리.
    """
    m = {f: "" for f in fields}
    for line in (text or "").splitlines():
        line = line.strip()
        if not line:
            continue
        if "\t" not in line:
            continue
        k, v = line.split("\t", 1)
        k = k.strip()
        v = v.strip()
        if k in m:
            m[k] = v
    return m

In [33]:
from rank_bm25 import BM25Okapi
import numpy as np
import re

def tokenize(text: str):
    return re.findall(r"[A-Za-z0-9가-힣]+", text.lower())

def build_bm25(chunks: list[str]):
    return BM25Okapi([tokenize(c) for c in chunks])

def retrieve_indices(
    retriever: str,
    query_text: str,
    chunks: list[str],
    index,
    embed_model,
    bm25,
    top_k: int,
    hybrid_cand_k: int,
):
    if retriever == "bm25":
        scores = bm25.get_scores(tokenize(query_text))
        idxs = np.argsort(scores)[::-1][:top_k]
        return [int(i) for i in idxs]

    if retriever == "dense":
        q_emb = embed_model.encode([query_text])
        _, I = index.search(q_emb.astype("float32"), top_k)
        return [int(i) for i in I[0]]

    if retriever == "hybrid":
        scores = bm25.get_scores(tokenize(query_text))
        cand = np.argsort(scores)[::-1][:hybrid_cand_k]
        cand = [int(i) for i in cand]

        cand_chunks = [chunks[i] for i in cand]
        cand_emb = embed_model.encode(cand_chunks)
        q_emb = embed_model.encode([query_text])

        cand_emb = cand_emb / (np.linalg.norm(cand_emb, axis=1, keepdims=True) + 1e-8)
        q_emb = q_emb / (np.linalg.norm(q_emb, axis=1, keepdims=True) + 1e-8)

        scores = (cand_emb @ q_emb.T).ravel()
        rerank = [cand[i] for i in np.argsort(scores)[::-1][:top_k]]
        return rerank

    raise ValueError(f"Unknown retriever: {retriever}")

def answer_batch(
    index, model, chunks, queries,
    retriever: str,
    bm25=None,
    top_k: int = 15,
    hybrid_cand_k: int = 100,
    llm_model: str = "gpt-5-mini",
    return_indices: bool = False,
):
    chunks_clean = [cleanup_chunk(c) for c in chunks]

    idxs_by_query = []
    contexts = []

    for field, q in queries:
        idxs = retrieve_indices(
            retriever=retriever,
            query_text=q,
            chunks=chunks_clean,
            index=index,
            embed_model=model,
            bm25=bm25,
            top_k=top_k,
            hybrid_cand_k=hybrid_cand_k,
        )
        idxs_by_query.append(idxs)
        contexts.append("\n\n---\n\n".join(chunks_clean[i] for i in idxs))

    context = "\n\n==========\n\n".join(
        [f"[{queries[i][0]}]\n{contexts[i]}" for i in range(len(queries))]
    )

    fields = [f for f, _ in queries]

    format_rule = (
        "아래 규칙을 반드시 지켜라.\n"
        "1) 출력은 정확히 field 개수만큼의 줄이다.\n"
        "2) 각 줄은 'field<TAB>answer' 형식이다. 탭(TAB) 필수.\n"
        "3) field는 아래 목록 중 하나만 사용한다.\n"
        "4) 설명/불릿/추가 문장/문서 원문 복사 금지.\n"
        "5) 문서에 답이 없으면 answer를 빈 문자열로 두되, 그 줄은 반드시 출력.\n"
        f"field 목록: {fields}\n"
        "예시:\n"
        "project_name\t2024년 ...\n"
        "agency\t벤처기업확인기관\n"
    )

    prompt = (
        RFP_PROMPT.format(questions=build_query_prompt(queries), context=context)
        + "\n\n"
        + format_rule
    )

    def _call(p):
        return client.chat.completions.create(
            model=llm_model,
            messages=[
                {"role": "system", "content": SYSTEM},
                {"role": "user", "content": p},
            ],
            max_completion_tokens=3000,
            reasoning_effort="low",
        )

    resp = _call(prompt)
    text = (resp.choices[0].message.content or "").strip()

    if not text:
        short_context = context[:7000]
        short_prompt = (
            RFP_PROMPT.format(questions=build_query_prompt(queries), context=short_context)
            + "\n\n"
            + format_rule
        )
        retry = _call(short_prompt)
        text = (retry.choices[0].message.content or "").strip()

    ans_map = parse_field_answers_relaxed(text, fields)
    answers = [ans_map.get(f, "") for f in fields]

    if return_indices:
        return answers, idxs_by_query
    return answers

## RAG 평가 (docs_api_summary 베이스라인 그대로 + gold로 점수 측정)

In [34]:
import json
import pandas as pd
from tqdm.auto import tqdm
from rapidfuzz import fuzz

In [35]:
# 평가 데이터 경로
print("BASE_DIR:", BASE_DIR)
print("RAW_DIR:", RAW_DIR, "exists:", RAW_DIR.exists())
print("EVAL_DIR:", EVAL_DIR, "exists:", EVAL_DIR.exists())

BASE_DIR: /home/ohs3201/codeit/codeit-part3-team4
RAW_DIR: /home/ohs3201/codeit/codeit-part3-team4/data/raw/files exists: True
EVAL_DIR: /home/ohs3201/codeit/codeit-part3-team4/data/raw/eval exists: True


### 평가용 정답 데이터 불러오기

In [36]:
def load_gold_fields_jsonl(path: Path) -> pd.DataFrame:
    rows = []
    with path.open("r", encoding="utf-8", errors="replace") as f:
        for line in f:
            if not line.strip():
                continue
            rows.append(json.loads(line))
    out = []
    for r in rows:
        iid = r["instance_id"]
        doc_id = r.get("doc_id", "")
        fields = r.get("fields", {}) or {}
        for k, v in fields.items():
            out.append({"instance_id": iid, "doc_id": doc_id, "field": k, "gold": v})
    return pd.DataFrame(out)


gold_evidence_df = pd.read_csv(GOLD_EVIDENCE_CSV)
gold_fields_df = load_gold_fields_jsonl(GOLD_FIELDS_JSONL)

display(gold_evidence_df.head(3))
display(gold_fields_df.head(3))

,instance_id,doc_id,page_start,page_end,anchor_text
0,G_Q001,(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf,4.0,4.0,사업개요
1,G_Q002,(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf,1.0,1.0,벤처기업확인기관
2,G_Q003,(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf,3.0,3.0,추진배경 및 방향


,instance_id,doc_id,field,gold
0,G_Q001,(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf,project_name,벤처확인종합관리시스템 기능 고도화
1,G_Q002,(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf,agency,벤처기업확인기관
2,G_Q003,(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf,purpose,"[복수의결권주식, 스톡옵션(주식매수선택권), 성과조건부주식교부계약(RS) 등의 기능..."


### Retrieval 평가(Anchor 기반)

`gold_evidence.csv`의 `anchor_text`가 top-k 청크 안에 포함되는지로 retrieval을 평가합니다.
- `recall@k`: top-k 안에서 anchor가 한 번이라도 나오면 1
- `mrr@k`: 첫 hit의 역순위(없으면 0)

In [37]:
def build_gold_anchor_map(df: pd.DataFrame) -> dict[str, list[str]]:
    m: dict[str, list[str]] = {}
    for _, r in df.iterrows():
        iid = str(r["instance_id"])
        anchor = str(r.get("anchor_text", "") or "").strip()
        if anchor:
            m.setdefault(iid, []).append(anchor)
    return m

GOLD_ANCHOR = build_gold_anchor_map(gold_evidence_df)


def eval_retrieval_by_anchor(chunks, idxs, anchors):
    # idxs가 [[...],[...]] 형태면 1차원으로 합치기(중복 제거)
    if len(idxs) and isinstance(idxs[0], list):
        flat = []
        for sub in idxs:
            flat.extend(sub)
        idxs = flat

    hit_rank = None
    for rank, ci in enumerate(idxs, start=1):
        if 0 <= int(ci) < len(chunks):
            c = chunks[int(ci)]
            if any(a in c for a in anchors):
                hit_rank = rank
                break

    recall = 1.0 if hit_rank is not None else 0.0
    mrr = 1.0 / hit_rank if hit_rank is not None else 0.0
    return {"recall": recall, "mrr": mrr}

In [38]:
# 질문 리스트: questions.csv에서 공통+문서별 질문을 합쳐 사용
import pandas as pd

questions_df = pd.read_csv(EVAL_DIR / "questions.csv")

def get_queries_for_doc(doc_name: str):
    common = questions_df[questions_df["doc_id"] == "*"][["type", "question"]]
    per_doc = questions_df[questions_df["doc_id"] == doc_name][["type", "question"]]
    merged = pd.concat([common, per_doc], ignore_index=True)
    return list(zip(merged["type"], merged["question"]))


### 정답 유사도 평가 함수

In [39]:
def eval_gen(pred: str, gold: str | None, threshold: int = 80) -> dict[str, float]:
    pred = (pred or "").strip()
    fill = 1.0 if pred and pred.lower() not in {"", "없음"} else 0.0
    if gold is None or str(gold).strip() == "":
        return {"fill": fill, "match": np.nan, "sim": np.nan}
    gold = str(gold).strip()
    sim = fuzz.token_set_ratio(pred, gold)
    return {"fill": fill, "match": 1.0 if sim >= threshold else 0.0, "sim": float(sim)}

In [40]:
SIM_THRESHOLD = 80
TOP_K = 15

RETRIEVERS = ["bm25", "dense", "hybrid"]
HYBRID_CAND_K = 100  # BM25로 뽑을 후보 수 (50~200 추천)

In [41]:
# docs_api_summary에서 만든 docs 리스트 사용
# docs = get_pdf_paths(RAW_DIR) 가 이미 실행되어 있어야 합니다.
DOC_PATHS = docs
for p in DOC_PATHS:
    print(" -", p.name)

 - (사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf
 - (사)부산국제영화제_2024년 BIFF & ACFM 온라인서비스 재개발 및 행사지원시.pdf
 - (사)한국대학스포츠협의회_KUSF 체육특기자 경기기록 관리시스템 개발.pdf
 - (재)예술경영지원센터_통합 정보시스템 구축 사전 컨설팅.pdf
 - 2025 구미 아시아육상경기선수권대회 조직위원회_2025 구미아시아육상경.pdf
 - BioIN_의료기기산업 종합정보시스템(정보관리기관) 기능개선 사업(2차).pdf
 - KOICA 전자조달_[긴급] [지문] [국제] 우즈베키스탄 열린 의정활동 상하원 .pdf
 - 경기도 안양시_호계체육관 배드민턴장 및 탁구장 예약시스템 구축 용역.pdf
 - 경기도 평택시_2024년도 평택시 버스정보시스템(BIS) 구축사업.pdf
 - 경기도사회서비스원_2024년 통합사회정보시스템 운영지원.pdf
 - 경상북도 봉화군_봉화군 재난통합관리시스템 고도화 사업(협상)(긴급).pdf
 - 경희대학교_[입찰공고] 산학협력단 정보시스템 운영 용역업체 선정.pdf
 - 고려대학교_차세대 포털·학사 정보시스템 구축사업.pdf
 - 고양도시관리공사_관산근린공원 다목적구장 홈페이지 및 회원 통합운영.pdf
 - 광주과학기술원_실시간통합연구비관리시스템(RCMS)  연계 모듈 변경 사업.pdf
 - 광주과학기술원_학사시스템 기능개선 사업.pdf
 - 국가과학기술지식정보서비스_통합정보시스템 고도화 용역.pdf
 - 국가철도공단_철도인프라 디지털트윈 정보화전략계획(ISP) 수립 용역(변.pdf
 - 국립인천해양박물관_국립인천해양박물관 해양자료관리시스템 구축 용.pdf
 - 국립중앙의료원_(긴급)「2024년도 차세대 응급의료 상황관리시스템 구축.pdf
 - 국민연금공단_2024년 이러닝시스템 운영 용역.pdf
 - 국민연금공단_사업장 사회보험료 지원 고시 개정에 따른 정보시스템 보.pdf
 - 국방과학연구소_기록관리시스템 통합 활용 및 보안 환경 구축.pdf
 - 국

### 문서 단위 평가

- generation: `gen_fill_rate`, `gen_match_rate`, `gen_avg_similarity`
- retrieval: `retrieval_recall@k`, `retrieval_mrr@k` (anchor 기반)
- score: 위 지표의 가중합

In [42]:
import unicodedata

def run_eval_for_doc(doc_path: Path) -> pd.DataFrame:
    """
    문서 1개에 대해:
    - 질문별 retrieval(top-k idxs_by_query) + generation(answers) 수행
    - gold_evidence(anchor_text) 기반 retrieval 평가(Recall@K, MRR@K)
    - gold_fields 기반 generation 평가(fill/match/sim)
    - retriever × generator 조합별 row 반환
    """
    doc_name = unicodedata.normalize("NFC", doc_path.name)

    # gold_fields에서 이 문서에 해당하는 평가 row 로드
    qdf = gold_fields_df[gold_fields_df["doc_id"].astype(str) == doc_name].copy()
    if len(qdf) == 0:
        print("⚠️ no gold fields:", doc_name)
        return pd.DataFrame()

    # 청킹/인덱싱
    index, chunks = pp.gen_input(doc_path, embed_model)
    bm25 = build_bm25(chunks)

    # 이 문서에서 평가할 (field, question) 쌍
    BASELINE_QUERIES = get_queries_for_doc(doc_name)  # list[tuple[field, question]]
    if not BASELINE_QUERIES:
        print("⚠️ no baseline queries:", doc_name)
        return pd.DataFrame()

    # field -> query position (질문별 idxs_by_query 매핑용)
    qpos = {f: i for i, (f, _) in enumerate(BASELINE_QUERIES)}

    rows = []
    for retriever in RETRIEVERS:
        for llm_model in GEN_MODELS:

            # 질문별 retrieval을 수행하는 answer_batch()가 return_indices=True일 때 idxs_by_query(list[list[int]])를 반환해야 함
            answers, idxs_by_query = answer_batch(
                index, embed_model, chunks, BASELINE_QUERIES,
                retriever=retriever,
                bm25=bm25,
                top_k=TOP_K,
                hybrid_cand_k=HYBRID_CAND_K,
                llm_model=llm_model,
                return_indices=True,
            )

            # field -> pred 답변
            answer_map = {
                k: (answers[i] if i < len(answers) else "")
                for i, (k, _) in enumerate(BASELINE_QUERIES)
            }

            r_list, g_list = [], []
            for _, row in qdf.iterrows():
                iid = str(row["instance_id"])
                field = str(row["field"])
                gold = row["gold"]
                pred = answer_map.get(field, "")

                anchors = GOLD_ANCHOR.get(iid, [])
                if anchors:
                    qi = qpos.get(field, None)
                    if qi is None or qi >= len(idxs_by_query):
                        r_list.append({"recall": np.nan, "mrr": np.nan})
                    else:
                        r_list.append(eval_retrieval_by_anchor(chunks, idxs_by_query[qi], anchors))
                else:
                    r_list.append({"recall": np.nan, "mrr": np.nan})

                g_list.append(eval_gen(pred, gold, threshold=SIM_THRESHOLD))

            rows.append({
                "doc_id": doc_name,
                "retriever": retriever,
                "generator": llm_model,
                "n_questions_total": int(len(qdf)),
                "n_questions_with_evidence": int(sum(1 for x in qdf["instance_id"] if str(x) in GOLD_ANCHOR)),
                "retrieval_recall@k": float(np.nanmean([x["recall"] for x in r_list])) if len(r_list) else np.nan,
                "retrieval_mrr@k": float(np.nanmean([x["mrr"] for x in r_list])) if len(r_list) else np.nan,
                "n_questions_with_gold_fields": int(len(qdf)),
                "gen_fill_rate": float(np.nanmean([x["fill"] for x in g_list])) if len(g_list) else np.nan,
                "gen_match_rate": float(np.nanmean([x["match"] for x in g_list])) if len(g_list) else np.nan,
                "gen_avg_similarity": float(np.nanmean([x["sim"] for x in g_list])) if len(g_list) else np.nan,
            })

    return pd.DataFrame(rows)

### 결과 집계 + 평균 테이블 생성

In [43]:
all_results = [run_eval_for_doc(p) for p in DOC_PATHS]
results_df = pd.concat([d for d in all_results if len(d)], ignore_index=True) if all_results else pd.DataFrame()

if len(results_df):
    results_df["score"] = (
        0.5 * results_df["retrieval_mrr@k"].fillna(0) +
        0.2 * results_df["retrieval_recall@k"].fillna(0) +
        0.3 * results_df["gen_match_rate"].fillna(0)
    )

display(results_df)  # 문서별 x 조합별 전체 값

if len(results_df):
    metric_cols = [
        "n_questions_total",
        "n_questions_with_evidence",
        "retrieval_recall@k",
        "retrieval_mrr@k",
        "n_questions_with_gold_fields",
        "gen_fill_rate",
        "gen_match_rate",
        "gen_avg_similarity",
        "score",
    ]
    summary_df = (
        results_df.groupby(["retriever", "generator"], as_index=False)[metric_cols]
        .mean(numeric_only=True)
    )
    display(summary_df)  # 평균표

if len(results_df):
    out_path = BASE_DIR / "outputs" / "eval_integrated_C3_ver1.csv"
    out_path.parent.mkdir(parents=True, exist_ok=True)
    results_df.to_csv(out_path, index=False, encoding="utf-8-sig")
    print("Saved:", out_path)

⚠️ no gold fields: BioIN_의료기기산업 종합정보시스템(정보관리기관) 기능개선 사업(2차).pdf
⚠️ no gold fields: KOICA 전자조달_[긴급] [지문] [국제] 우즈베키스탄 열린 의정활동 상하원 .pdf
⚠️ no gold fields: 고려대학교_차세대 포털·학사 정보시스템 구축사업.pdf
⚠️ no gold fields: 기초과학연구원_2025년도 중이온가속기용 극저온시스템 운전 용역.pdf
⚠️ no gold fields: 부산관광공사_경영정보시스템 기능개선.pdf
⚠️ no gold fields: 사단법인 보험개발원_실손보험 청구 전산화 시스템 구축 사업.pdf
⚠️ no gold fields: 사단법인아시아물위원회사무국_우즈벡-키르기즈스탄 기후변화대응 스.pdf
⚠️ no gold fields: 서민금융진흥원_서민금융진흥원 서민금융 채팅 상담시스템 구축.pdf
⚠️ no gold fields: 서영대학교 산학협력단_전문대학 혁신지원사업 서영대학교 차세대 교육.pdf
⚠️ no gold fields: 서울시립대학교_[사전공개] 학업성취도 다차원 종단분석 통합시스템 1차.pdf
⚠️ no gold fields: 서울특별시 여성가족재단_(재공고, 협상) 서울 디지털성범죄 안심지원센.pdf
⚠️ no gold fields: 서울특별시_2024년 지도정보 플랫폼 및 전문활용 연계 시스템 고도화 용.pdf
⚠️ no gold fields: 서울특별시교육청_서울특별시교육청 지능정보화전략계획(ISP) 수립(2차) .pdf
⚠️ no gold fields: 세종테크노파크_세종테크노파크 인사정보 전산시스템 구축 용역 입찰공.pdf
⚠️ no gold fields: 수협중앙회_강릉어선안전조업국 상황관제시스템 구축.pdf
⚠️ no gold fields: 수협중앙회_수협중앙회 수산물사이버직매장 시스템 재구축 ISMP 수립 입.pdf
⚠️ no gold fields: 울산광역시_2024년 버스정보시스템 확대 구축 및 기능개선 용역.pdf
⚠

,doc_id,retriever,generator,n_questions_total,n_questions_with_evidence,retrieval_recall@k,retrieval_mrr@k,n_questions_with_gold_fields,gen_fill_rate,gen_match_rate,gen_avg_similarity,score
0,(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf,bm25,gpt-5-mini,21,21,0.809524,0.530385,21,0.952381,0.190476,49.467545,0.484240
1,(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf,bm25,gpt-5-nano,21,21,0.809524,0.530385,21,0.000000,0.000000,0.000000,0.427098
2,(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf,dense,gpt-5-mini,21,21,0.857143,0.517063,21,0.952381,0.238095,52.942905,0.501389
3,(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf,dense,gpt-5-nano,21,21,0.857143,0.517063,21,1.000000,0.380952,48.813552,0.544246
4,(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf,hybrid,gpt-5-mini,21,21,0.857143,0.515873,21,0.952381,0.190476,51.537073,0.486508
...,...,...,...,...,...,...,...,...,...,...,...,...
175,문화체육관광부 국립민속박물관_2024년 국립민속박물관 민속아카이브 자.pdf,bm25,gpt-5-nano,21,21,0.904762,0.637698,21,0.000000,0.000000,0.000000,0.499802
176,문화체육관광부 국립민속박물관_2024년 국립민속박물관 민속아카이브 자.pdf,dense,gpt-5-mini,21,21,1.000000,0.815476,21,0.952381,0.428571,67.540564,0.736310
177,문화체육관광부 국립민속박물관_2024년 국립민속박물관 민속아카이브 자.pdf,dense,gpt-5-nano,21,21,1.000000,0.815476,21,1.000000,0.380952,53.716022,0.722024
178,문화체육관광부 국립민속박물관_2024년 국립민속박물관 민속아카이브 자.pdf,hybrid,gpt-5-mini,21,21,1.000000,0.815476,21,0.952381,0.428571,69.983694,0.736310


,retriever,generator,n_questions_total,n_questions_with_evidence,retrieval_recall@k,retrieval_mrr@k,n_questions_with_gold_fields,gen_fill_rate,gen_match_rate,gen_avg_similarity,score
0,bm25,gpt-5-mini,21.0,21.0,0.815952,0.513606,21.0,0.938095,0.258730,51.080087,0.497613
1,bm25,gpt-5-nano,21.0,21.0,0.815952,0.513606,21.0,0.788889,0.206349,36.770794,0.481898
2,dense,gpt-5-mini,21.0,21.0,0.870794,0.647122,21.0,0.949206,0.260317,52.414303,0.575815
3,dense,gpt-5-nano,21.0,21.0,0.870794,0.647122,21.0,0.811111,0.190476,35.605119,0.554863
4,hybrid,gpt-5-mini,21.0,21.0,0.869127,0.646834,21.0,0.941270,0.260317,52.785005,0.575338
5,hybrid,gpt-5-nano,21.0,21.0,0.869127,0.646834,21.0,0.812698,0.211111,38.477607,0.560576


Saved: /home/ohs3201/codeit/codeit-part3-team4/outputs/eval_integrated_C3_ver1.csv


### 질문 샘플 및 답변

In [44]:
# 질문 샘플 + 모델 답변 보기 (문서 1개 기준)
doc_name = results_df.sort_values("gen_match_rate").iloc[0]["doc_id"]  # 예시: 성능 낮은 문서
sample_n = 5

# 문서 선택
doc_path = next(p for p in DOC_PATHS 
                if unicodedata.normalize("NFC", p.name) == unicodedata.normalize("NFC", doc_name))


# 준비
qdf = gold_fields_df[gold_fields_df["doc_id"].astype(str) == doc_name].copy()
BASELINE_QUERIES = get_queries_for_doc(doc_name)

index, chunks = pp.gen_input(doc_path, embed_model)
model = embed_model

bm25 = build_bm25(chunks)

# 배치 답변
answers, idxs = answer_batch(
    index, embed_model, chunks, BASELINE_QUERIES,
    retriever="bm25",          # 또는 "dense", "hybrid"
    bm25=bm25,
    top_k=TOP_K,
    hybrid_cand_k=HYBRID_CAND_K,
    llm_model="gpt-5-mini",    # 또는 "gpt-5-nano"
    return_indices=True
)
answer_map = {k: answers[i] if i < len(answers) else "" for i, (k, _) in enumerate(BASELINE_QUERIES)}

# 샘플 출력
rows = []
for _, row in qdf.head(sample_n).iterrows():
    field = str(row["field"])
    gold = row["gold"]
    pred = answer_map.get(field, "")
    question = dict(BASELINE_QUERIES).get(field, "")
    rows.append({"field": field, "question": question, "gold": gold, "pred": pred})

display(pd.DataFrame(rows))

,field,question,gold,pred
0,project_name,사업(용역)명은 무엇인가?,벤처확인종합관리시스템 기능 고도화,2024년 「벤처확인종합관리시스템 기능 고도화」 용역사업
1,agency,발주 기관(수요기관)은 어디인가?,벤처기업확인기관,벤처기업확인기관
2,purpose,사업 목적(추진 배경)은 무엇인가?,"[복수의결권주식, 스톡옵션(주식매수선택권), 성과조건부주식교부계약(RS) 등의 기능...",복수의결권주식·스톡옵션·성과조건부주식 업무시스템 고도화 및 스톡옵션 데이터 이관·구축
3,budget,총 사업 예산(사업비)은 얼마인가?,"352,000,000원","352,000,000원(부가가치세 포함)"
4,contract_type,계약 방식(일반경쟁/제한경쟁/협상에 의한 계약 등)은 무엇인가?,제한경쟁입찰,제한경쟁입찰(협상에 의한 계약)
